In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
from scipy.stats import ttest_ind 

file_path = "data/Building_Energy_Benchmarking.csv"
data = pd.read_csv(file_path)

print("Data Shape:", data.shape)
print("Columns:\n", data.columns)
print("Data Types:\n", data.dtypes)
print("Number of Missing Values From Each Columns:\n", data.isnull().sum())

threshold = 0.4 * len(data) # Setting threshold for 40%

data_clean1 = data.dropna(axis=1, thresh=threshold) # Dropping columns that exceed the threshhold

numerical_columns = data_clean1.select_dtypes(include=['int64', 'float64']).columns #Defining a numerical column

for col in numerical_columns:
    data_clean1[col].fillna(data_clean1[col].median(), inplace = True)  #Filling numerical columns with median

category_columns = data_clean1.select_dtypes(include=['object']).columns #Defining a categorical column

for col in category_columns:
    data_clean1[col].fillna(data_clean1[col].mode(), inplace= True) #Filling categorical columns with mode

def extract_num(column):
    return column.str.extract(r"(\d+\.?\d*)").astype(float)

unit_columns = [col for col in data_clean1.columns if re.search(r"\(.*\)", col)] #Comprehension to loop through columns in dataset to see if they have () for units

for col in unit_columns:
    data_clean1[col] = extract_num(data_clean1[col].astype(str)) #Apply regex numerical extraction on columns with units

def postal_code_clean(post_code): #Regex to clean postal codes
    match = re.match(r"([A-Za-z]\d[A-Za-z])\s?(\d[A-Za-z]\d)", str(post_code))
    return match.group(1) + " " + match.group(2) if match else np.nan

data_clean1["Postal Code"] = data_clean1["Postal Code"].apply(postal_code_clean)

def clean_text(column): #Function to clean text in Property Names and Addresses
    column = column.str.replace(r"\b(calgary|alberta)\b", "", flags=re.IGNORECASE, regex=True) # Following rows ensure quandrants are upper case and remove Calgary and Alberta from address 1 
    column = column.str.replace(r"\b(ne|nw|se|sw)\b", lambda x: x.group(0).upper(), flags=re.IGNORECASE, regex=True)
    column = column.str.title().str.strip()
    return column
data_clean1["Property Name"] = clean_text(data_clean1["Property Name"].astype(str)) #Applying functions on both columns
data_clean1["Address 1"] = clean_text(data_clean1["Address 1"].astype(str))


summary = data_clean1.describe()
print("\nSummary Statistics:\n", summary)

energy_use_int_avg = data_clean1.groupby("Primary Property Type - Self Selected")["Site EUI (GJ/m²)"].mean().sort_values(ascending=False)

ghg_by_year = data_clean1.groupby("Year Ending")["Total GHG Emissions (Metric Tons CO2e)"].sum()

top5_energy = data_clean1.nlargest(5, "Site Energy Use (GJ)")

def clean_ghg_total(value):
    match = re.findall(r"(\d+\.?\d*)", str(value))
    return float(match[0]) if match else np.nan #Function to clean numerical values with regex 

data_clean1['Total GHG Emissions (Metric Tons CO2e)'] = data_clean1['Total GHG Emissions (Metric Tons CO2e)'].apply(clean_ghg_total)

for name, group in data_clean1.groupby("Primary Property Type - Self Selected"): #IQR detection of outliers in GHG column
    Q1 = group["Total GHG Emissions (Metric Tons CO2e)"].quantile(0.25)
    Q3 = group["Total GHG Emissions (Metric Tons CO2e)"].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

outliers = (group["Total GHG Emissions (Metric Tons CO2e)"] < lower_bound) | (group["Total GHG Emissions (Metric Tons CO2e)"] > upper_bound)
median_value = group["Total GHG Emissions (Metric Tons CO2e)"].median()
data_clean1.loc[outliers.index, "Total GHG Emissions (Metric Tons CO2e)"] = median_value # Filtering detected outliers with median value

eui_trend = data_clean1.groupby("Year Ending")["Site EUI (GJ/m²)"].mean()
# Time Series Analysis
plt.figure(figsize=(12,6)) 
plt.plot(eui_trend.index, eui_trend.values, color='blue')
plt.title("Yearly Trend of Average Site EUI (GJ/m²)")
plt.xlabel("Year")
plt.ylabel("Average Site EUI (GJ/m²)")
plt.xticks(ticks=eui_trend.index, labels=eui_trend.index.astype(int))
plt.show()

top_10 = data_clean1.groupby(['Property Id', 'Property Name'])["Total GHG Emissions (Metric Tons CO2e)"].sum().sort_values(ascending=False).head(10).reset_index() #Filtering top 10 highest GHG emissions by property name
# Bar Chart
figure, ax = plt.subplots()
bars = ax.bar(top_10['Property Name'], top_10["Total GHG Emissions (Metric Tons CO2e)"]) 
plt.title("Top 10 Buildings with Highest GHG Emissions")
plt.xlabel("Building Name")
plt.ylabel("Total GHG Emissions (Metric Tons CO2e)")
plt.xticks(rotation=45, ha="right")
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, f"{height:.1f}", ha="center", va="bottom")
plt.show()

#Heatmap Data
heatmap_data = data_clean1.pivot_table(index="Primary Property Type - Self Selected", values="Site EUI (GJ/m²)", aggfunc="mean") #Utilizing a pivot table to get the proper data compiled for the heatmap

plt.figure(figsize=(12,6))
sns.heatmap(heatmap_data, annot=True, cmap="YlGnBu", fmt=".2f", linewidths=0.5)
plt.title("Heatmap of Average Site EUI (GJ/m²) by Property Type")
plt.xlabel("Property Type")
plt.ylabel("Average Site EUI (GJ/m²)")
plt.show()
#Correlation Matrix
correlation = ["Site Energy Use (GJ)", "Total GHG Emissions (Metric Tons CO2e)", "Property GFA - Self-Reported (m²)"]
corr_matrix = data_clean1[correlation].corr()
display(corr_matrix)

print(data["Primary Property Type - Self Selected"].unique())
# To use Energy Star Score column in dataset, we have to revert back to pre-dropped data as the column contained more than 40% NAN values
office = data[data["Primary Property Type - Self Selected"] == "Office"]["ENERGY STAR Score"].dropna()
museum = data[data["Primary Property Type - Self Selected"] == "Museum"]["ENERGY STAR Score"].dropna()

t_stat, p_value = ttest_ind(office, museum)

print("T-Stat:\n", t_stat, "\nP Value:\n", p_value)




           






